In [ ]:
!python --version
!nvcc --version

# Installer l'extension CUDA pour Jupyter
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpc32n7i35".


In [ ]:
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2024-12-30 20:36:37--  https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 283010 (276K) [text/plain]
Saving to: ‘stb_image.h’

stb_image.h         100%[===================>] 276.38K  --.-KB/s    in 0.02s   

2024-12-30 20:36:37 (11.5 MB/s) - ‘stb_image.h’ saved [283010/283010]

--2024-12-30 20:36:37--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_

In [5]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/stb_image.h"
#include "/content/stb_image_write.h"


unsigned char* resize_image(unsigned char* input_image, int width, int height, int new_width, int new_height) {
    unsigned char* resized_image = (unsigned char*)malloc(new_width * new_height * sizeof(unsigned char));
    float x_ratio = (float)width / new_width;
    float y_ratio = (float)height / new_height;

    for (int y = 0; y < new_height; y++) {
        for (int x = 0; x < new_width; x++) {
            int px = (int)(x * x_ratio);
            int py = (int)(y * y_ratio);
            resized_image[y * new_width + x] = input_image[py * width + px];
        }
    }
    return resized_image;
}

__global__ void thresholding(unsigned char *image, int n, unsigned char *imageResult) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    if (idx < n) {
        imageResult[idx] = (image[idx] >= 188) ? 255 : 0;
    }
}

int main() {
    int width, height, channels;
    unsigned char *image = stbi_load("/content/images.png", &width, &height, &channels, 1);
    if (image == NULL) {
        printf("Erreur lors du chargement de l'image\n");
        return 1;
    }

    int factors[] = {1, 3, 5, 7, 9, 11};
    int num_factors = sizeof(factors) / sizeof(factors[0]);

    unsigned char *d_image, *d_imageResult;
    printf("Taille originale de l'image : %dx%d\n", width, height);
    printf("Facteur\tTemps moyen du kernel (ms)\n");

    for (int i = 0; i < num_factors; i++) {
        int factor = factors[i];
        int new_width = width / factor;
        int new_height = height / factor;
        int n = new_width * new_height;

        unsigned char* resized_image = resize_image(image, width, height, new_width, new_height);

        cudaMalloc((void**)&d_image, n * sizeof(unsigned char));
        cudaMalloc((void**)&d_imageResult, n * sizeof(unsigned char));
        cudaMemcpy(d_image, resized_image, n * sizeof(unsigned char), cudaMemcpyHostToDevice);

        int blockSize = 64;
        int numBlocks = (n + blockSize - 1) / blockSize;

        // Mesure du temps d'exécution
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);

        thresholding<<<numBlocks, blockSize>>>(d_image, n, d_imageResult);
        cudaDeviceSynchronize();

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);



        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);

        printf("Facteur : %d, Temps moyen : %.3f ms\n", factor, milliseconds);

        // Récupération de l'image segmentée sur le CPU
        unsigned char* h_imageResult = (unsigned char*)malloc(n * sizeof(unsigned char));
        cudaMemcpy(h_imageResult, d_imageResult, n * sizeof(unsigned char), cudaMemcpyDeviceToHost);

        // Sauvegarde de l'image segmentée
        char filename[50];
        sprintf(filename, "/content/segmented_factor_%d.png", factor);
        stbi_write_png(filename, new_width, new_height, 1, h_imageResult, new_width);

        // Nettoyage
        free(resized_image);
        free(h_imageResult);
        cudaFree(d_image);
        cudaFree(d_imageResult);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    // Libération de l'image originale
    stbi_image_free(image);

    return 0;
}

Taille originale de l'image : 225x225
Facteur	Temps moyen du kernel (ms)
Facteur : 1, Temps moyen : 0.181 ms
Facteur : 3, Temps moyen : 0.018 ms
Facteur : 5, Temps moyen : 0.016 ms
Facteur : 7, Temps moyen : 0.016 ms
Facteur : 9, Temps moyen : 0.015 ms
Facteur : 11, Temps moyen : 0.016 ms
Processus terminé.

